In [1]:
 %pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from pylab import *
from scipy import *
from scipy import stats, io
import numpy as np
import struct
import tables as tb
from attrdict import AttrDict
import matplotlib.pyplot as plt
import os as os
from phy.io import KwikModel
import codecs as codecs
import matplotlib.gridspec as gridspec
import matplotlib.ticker as mtick

In [3]:
#----------------------------------------------------------------------------------------
#SAVING BINARY OBJECTS DATA
#need to automate data folder creation
#----------------------------------------------------------------------------------------
import pickle 

def save_obj(obj, name ):
    with open( name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open( name + '.pkl', 'rb') as f:
        return pickle.load(f)

# FUNCTIONS TO BUILD PSTHs

In [108]:
#---------------------------------------------------------
# GET STIMTIMES
#---------------------------------------------------------
def get_stimtimes(stimfile,timefile):
    
    stims = open(stimfile, 'r')
    times = open(timefile, 'r')
    
    stimdata = stims.read().splitlines()
    timedata = times.read().splitlines()
   
    stims.close()
    times.close()
    
    return stimdata, timedata

#---------------------------------------------------------
# BUILD DICT STIM, reads stims and times of .txt files of experiments
#---------------------------------------------------------
def build_dict_stim(stimfile, timefile):

    stims, times = get_stimtimes(stimfile,timefile)
    #------------------------------------------
    # count number of episodes
    episodes=1
    stims_ep=0
    for time in times[:-1]:             #last line is blank, we skip it
        if time == '': 
            episodes+=1
        if episodes ==1:
            stims_ep+=1
    print('Total episodes: ', episodes)        
    print('Total stims per episode: ', stims_ep)        
    #------------------------------------------
    # starting times of each stimulus
    starts = np.zeros([episodes, stims_ep])
    tot_stims = episodes*stims_ep

    st_count=0
    for t in times[:-1]:
        if t!='':
            st = st_count % stims_ep
            ep = st_count // stims_ep 
            starts[ep,st] = t
            st_count+=1
    #------------------------------------------
    # stims        
    st_pad = zeros([episodes,stims_ep]) # 1 row, 2 arc, 3 pad
    st_type = zeros([episodes,stims_ep]) # 1 hold, 2 pass, 3 release        
    st_isi = zeros([episodes,stims_ep]) # 10 , 20, 50
    st_rep = zeros([episodes,stims_ep]) # 2, 5, 10
    st_ctrl = zeros([episodes,stims_ep]) # 0, blank, 1 ctrl1, 2 ctrl2, 3 ctrl3, 10 normal 
    # dictionary of stims
    st_logic = {} 
    st_logic['pad'] = {'ROW' : 1, 'ARC' : 2, 'PAD' : 3}
    st_logic['types'] = {'hold' : 1, 'pass' : 2, 'release' : 3}
    st_logic['ctrl'] = {'BLANK' : 0, 'Ctrl1' : 1, 'Ctrl2' : 2, 'Ctrl3' : 3, 'Normal' : 10}

    ep=0
    st=0
    # get stims
    for stim in stims[:-3]:           #last three lines useless
        line = stim.split()
        if line and ep<episodes:

            if line[1]=='BLANK':
                st+=1
                if st%stims_ep==0:
                    st=0
                    ep+=1
            elif len(line)>3:
                if line[1][1:4]=='ROW': st_pad[ep,st]=1
                elif line[1][1:4]=='ARC': st_pad[ep,st]=2
                elif line[1][1:4]=='PAD': st_pad[ep,st]=3

                if line[1][5:-1]=='Hold': st_type[ep,st]=1
                elif line[1][5:-1]=='Pass': st_type[ep,st]=2
                elif line[1][5:-1]=='Release': st_type[ep,st]=3

                if line[3]=='REP_2': st_rep[ep,st]=2
                elif line[3]=='REP_5': st_rep[ep,st]=5
                elif line[3]=='REP_10': st_rep[ep,st]=10

                if line[5]=='ISI_10': st_isi[ep,st]=10
                elif line[5]=='ISI_20': st_isi[ep,st]=20
                elif line[5]=='ISI_50': st_isi[ep,st]=50
                    
                if line[7]=='Normal': st_ctrl[ep,st]=10
                elif line[7][0:5]=='Ctrl1': st_ctrl[ep,st]=1
                elif line[7][0:5]=='Ctrl2': st_ctrl[ep,st]=2
                elif line[7][0:5]=='Ctrl3': st_ctrl[ep,st]=3
                elif line[7][0:5]=='Ctrl4': st_ctrl[ep,st]=4

                st+=1
                if st%stims_ep==0:
                    st=0
                    ep+=1

    Stim_dict = AttrDict({'st_logic':st_logic, 'episodes': episodes  ,'stims_ep':stims_ep  ,'st_times': starts})
    Stim_dict.update({'st_isi':st_isi,'st_rep':st_rep,'st_type':st_type,'st_ctrl': st_ctrl,'st_pad':st_pad})
    
    return Stim_dict

#----------------------------------------------------------------------------------------
# BUILD HIST DICTIONARY
#----------------------------------------------------------------------------------------
import copy as cp

def build_hist_dict():
    isis = [10,20,50]
    reps = [2,5,10]
    pads = ['arc','row','pad']
    types = ['hold','pas','release']
    
    Isis ={}
    for i in isis:
        Isis[i]= []
    
    Reps ={}
    for r in reps:
        Reps[r] = cp.deepcopy(Isis)
    
    Pads = {}
    for p in pads:
        Pads[p] = cp.deepcopy(Reps)
    
    Types = {}
    for t in types:
        Types[t] = cp.deepcopy(Pads)
    
    Normal = cp.deepcopy(Types)
    Ctrl1 = cp.deepcopy(Types)
    Ctrl2 = cp.deepcopy(Types)
    Ctrl3 = cp.deepcopy(Types)
    Ctrl4 = cp.deepcopy(Types)
    
    counts_n = cp.deepcopy(Normal)
    counts_c1 = cp.deepcopy(Normal)
    counts_c2 = cp.deepcopy(Normal)
    counts_c3 = cp.deepcopy(Normal)
    counts_c4 = cp.deepcopy(Normal)
    
    Counts ={}
    Counts['Normal'] = counts_n
    Counts['Ctrl1'] = counts_c1
    Counts['Ctrl2'] = counts_c2
    Counts['Ctrl3'] = counts_c3
    Counts['Ctrl4'] = counts_c4
    Counts['Blank'] = []
    
    hist_logic = 'Type x Pad x Rep x Isi'
    hist = AttrDict({ 'Blank' :[],  'Normal': Normal, 'Ctrl1':Ctrl1 , 'Ctrl2':Ctrl2 , 'Ctrl3': Ctrl3, 'Ctrl4': Ctrl4 , 'hist_logic': hist_logic, 'Counts':Counts})

    return hist
    
#----------------------------------------------------------------------------------------
# READKWIKINFO
#----------------------------------------------------------------------------------------
# We read the data of the output from klusterkwik: spike times and cluster-number of each
# cluster-number is in klustaviewa series (can be as high as 130 e.g.)
# Grupete stands for cluster groups! 2: good clusters, 1: multiunits, 0: unsorted, 3: noise
def readkwikinfo(kwik, grupete=2):
    model = KwikModel(kwik) # load kwik model from file
    spiketimes = model.spike_times # extract the absolute spike times
    clusters = model.cluster_groups # extract the cluster names
    sample_rate = model.sample_rate # extract sampling freq
    
    spikedata = {} # initialise dictionary
    for cluster in clusters.keys():
        clustergroup = clusters[cluster]
        if clustergroup==grupete: # only look at specified type of cluster, 0 = noise, 1 = MUA, 2 = GOOD, 3 = unsorted
            spiketimematrix = AttrDict({'spike_times': np.zeros(len(spiketimes[where(model.spike_clusters==cluster)]))})
            spiketimematrix.spike_times = spiketimes[where(model.spike_clusters==cluster)]
            spikedata[cluster] = spiketimematrix # data structure is a dictionary with attribute accessible spiketimes
            # attribute accessible means that spikedata.spike_times works, normal dictionaries would be spikedata[spike_times]
    
    model.close()
    
    return spikedata, sample_rate

#----------------------------------------------------------------------------------------
# BUILDS PSTH
#----------------------------------------------------------------------------------------
def BuildPSTH(Stims, Spikes, sampling_freq, exp, meas):
    
    stimtimes = {}
    stim_samp = 1/.0009997575757
    # make an 'output dict'
    # the PSTH will be built on -tbefore:tafter
    PSTH_times = {}
    
    # Loop each neuron and get the spikes.
    for neuron in list(Spikes.keys()): 
        codename = 'exp'+ str(exp) + '_' + str(meas) + '_c' + str(neuron)
        psth = AttrDict({'clusnum': neuron,'exp' : int(exp) , 'meas': int(meas[1]) , 'shank': int(meas[3])})
        psth.update(AttrDict({'psth_counts': [] , 'psth_times': []}))
        
        hist= build_hist_dict()
        
        #loop episodes and stims_per_episode, and populate the histograms
        for ep in arange(Stim.episodes):
            for se in arange(Stim.stims_ep):
                c = Stims.st_ctrl[ep][se]
                t = Stims.st_type[ep][se]
                p = Stims.st_pad[ep][se]
                r = Stims.st_rep[ep][se]
                i = Stims.st_isi[ep][se]
                start = Stims.st_times[ep][se]
                if c=='Blank':
                    t_after=0.5
                    hist[c].append (Spikes[(start <= spikes) * (spikes <= times + t_after)])
                    hist['Counts'][c] += len(Spikes[(start <= spikes) * (spikes <= times + t_after)])
                else:
                    t_after = 0.5*r
                    hist[c][t][p][r][i].append(Spikes[(start <= spikes) * (spikes <= times + t_after)])
                    hist['Counts'][c][t][p][r][i] += lenSpikes([(start <= spikes) * (spikes <= times + t_after)])
                                                       
        PSTH_times[codename] = hist
       
    return PSTH_times


In [106]:
Folder = '/media/matias/DATA/WORKSPACE2/EXP_2'        
stimfile = Folder +'/EXPECT-151217-stims-7.txt'   
timefile = Folder +'/EXPECT-151217-times-7.txt'   

Stims = build_dict_stim(stimfile, timefile)

print(Stims.keys())

h = build_hist_dict()

print(h.keys())
print(h.hist_logic)

print(Stims.st_logic)

Stims.st_rep.shape


Total episodes:  15
Total stims per episode:  28
dict_keys(['st_isi', 'episodes', 'st_rep', 'st_times', 'st_ctrl', 'st_type', 'stims_ep', 'st_pad', 'st_logic'])
dict_keys(['Blank', 'Ctrl1', 'hist_logic', 'Ctrl2', 'Normal', 'Ctrl3', 'Ctrl4', 'Counts'])
Type x Pad x Rep x Isi
AttrDict({'types': {'pass': 2, 'hold': 1, 'release': 3}, 'pad': {'PAD': 3, 'ROW': 1, 'ARC': 2}, 'ctrl': {'Normal': 10, 'BLANK': 0, 'Ctrl2': 2, 'Ctrl1': 1, 'Ctrl3': 3}})


(15, 28)

In [24]:
for m in arange(10)+1:
    M[i] = ['m'+str(m)+'s'+str(s) for s in arange(8)+1]

M

['m10s1', 'm10s2', 'm10s3', 'm10s4', 'm10s5', 'm10s6', 'm10s7', 'm10s8']

# Experiment Files and Folders

In [5]:
# In this cell you put all the information to make the code portable from computer to computer
# You have to place all the file names and experiments, then you loop whichever you want to analyse
#--------------------------------------------------------------------------------
#Experiment numbers
ExpeNum = [1,2]

#--------------------------------------------------------------------------------
#Folders for measurements and experiments (this is how we separate shanks in folders for individual analyses)

M = 'm

m164 = ['m1s'+s for s in arange(8)+1]
m264 = ['m2s1','m2s2','m2s3','m2s4','m2s5','m2s6','m2s7','m2s8']
m364 = ['m3s1','m3s2','m3s3','m3s4','m3s5','m3s6','m3s7','m3s8']
m464 = ['m4s1','m4s2','m4s3','m4s4','m4s5','m4s6','m4s7','m4s8']

#--------------------------------------------------------------------------------
#Kwik files    

files20 = [ 'MEAS-150707-1_ele01_ele08.kwik',
            'MEAS-150707-1_ele09_ele16.kwik',
            'MEAS-150707-1_ele17_ele24.kwik',
            'MEAS-150707-1_ele25_ele32.kwik',
            'MEAS-150707-23_ele01_ele08.kwik',
            'MEAS-150707-23_ele16_ele09.kwik']#,
            #'MEAS-150707-23_ele17_ele24.kwik',  not in S2
            #'MEAS-150707-23_ele25_ele32.kwik',] not in S2

#--------------------------------------------------------------------------------
#--------------------------------------------------------------------------------
# Here I create my dictionary of experiments
Expe={}
Vtags={}
Stim={}
for num in ExpeNum: 
    Expe[num] = dict()
    Vtags[num] = dict()
#---------------------------------------
i=0        
for meas in np.append(m164[0:4],m364[0:2]):
    Expe[20][meas] = files20[i]
    i+=1
#---------------------------------------

    i+=1

#--------------------------------------------------------------------------------
#Vtag files 
Vtags[20] = ['MEAS-150707-1_Vtag1.dat','nada','MEAS-150707-23_Vtag1.dat']

   
#--------------------------------------------------------------------------------
#Root folder to work in, such all will be in subfolders 
#e.g.: "/EXP_23/m1s1/" for data or "/STIM/" for stims
rootF = '/home/matias/WORKSPACE/'    
stimFolder = rootF +'STIM/'



# Define and load data files from experiments

In [7]:
global binname, textname
#---------------------------------------------------------------------------------------
SelExp = [1]                         #select experiment numbers!
grupete = [2]   #select cluster groups! 2 for good clusters 1 for multiunits, 3 for unsorted

#select measurement and/or shanks!
#Measurements = m264[7:8]           #['m1s1']#['m3s1','m3s3']#m12[-4:]#['m1s1','m1s2','m1s3','m1s4']   

#select type of stimuli, for PSTH is only 'F' and this does not change anything
#choices = ['F','C','U']                      #select stimulus type (for STA and STC)

dirs =[]
#--------------------------------------------------------------------------------
# Loop Experiments
#--------------------------------------------------------------------------------
last_exp=0     #we use this to load stim only when we change experiment
for expe in SelExp:
    
    PSTHdata = {}
    PSTH_spikes_counts = {}
    
    Measurements = sorted(Expe[expe])                         #uncommento to select all
    print(expe)

    #This two lines are to account for diffrerent stims when looping diffrerent experiments
    stimfile = stimFolder + Stim[expe] 
    timefile = stimFolder + Times[expe] 
        
    Stims = build_dict_stim(stimfile, timefile)
    
    last_meas =0   #we use this to find when we change measurement to load Vtag and stim again
    
    #--------------------------------------------------------------------------------
    #loop goodunits, multiunits, unsorted...
    for group in grupete:   #2 for good clusters 1 for multiunits 3 for unsorted
        #folder names
        if group ==3:
            dirs  = [rootF + 'OUTPUT/EXP_'+str(expe)]
        if group ==2:
            dirs  = [rootF + 'OUTPUT/EXP_'+str(expe)]
        if group ==1:
            dirs  = [rootF + 'OUTPUT/EXP_'+str(expe)]
        #--------------------------------------------------------------------------------
        #create output folders
        for dir in dirs:
            if not os.path.exists(dir):
                os.makedirs(dir) 
        dire = dirs[0] +'/'
        titles = 'Exp'+ str(expe) + '_Meas_' + meas[1] + '_Shank_' + meas[3]
        #--------------------------------------------------------------------------------
        print('group:', group)
        #--------------------------------------------------------------------------------
        #loop measurements and shanks
        #Measurements = sorted((Expe[expe]))
        
        for meas in Measurements:           
            print(meas)
            current_meas = int(meas[1])   #measurement number
            #---------------------------------------------------------------
            #select datafile
            sp_file = rootF_kwiks + 'EXP_' + str(expe) +'/Spike_Sort/'+ meas +'/'+ Expe[expe][meas]
            #load datafile
            Spikes, sampling_freq = readkwikinfo(sp_file, group)  
                    
            if len(Spikes.keys())>0:                              #do only if there are clusters
                #Build PSTHs
                print('   building psths')
    
                PSTH_data = BuildPSTH(Stims, Spikes, sampling_freq, exp, meas) :
               
    print('   saving')
    if group == 2:
        filesave =rootF +'Expect_git/data/'+'psthdata' + str(expe)
    else:
        filesave =rootF +'S2_git/data/'+ 'psthdataMultiR' + str(expe)
        
    save_obj(PSTHdata,filesave)

22
group: 2
m1s1
   reading stim at: m1s1
   trimming stim at: m1s1
m1s2
   building psths
m1s3
   building psths
m1s4
   building psths
m3s1
   reading stim at: m3s1
   trimming stim at: m3s1
   building psths
m3s2
   building psths
m3s3
   building psths
m3s4
   building psths
   saving


In [34]:
#filesave

sorted(list(PSTHdata.keys()))
PSTHdata['exp20_m1s1_c29'].keys()

data = rootF +'S2_git/data/'+'psthdata' + str(22)

a = load_obj(data)

a['exp22_m3s4_c1138'].keys()

dict_keys(['psth_counts', 'meas', 'psth_length', 'clusnum', 'psth_times', 'exp', 'shank'])

In [ ]:
#update spiketimes to wavedataAll
SelExp = [20] #[22,24,26,27,28,29,30,31,32] #Expe                                        #select experiment numbers!

Folder = '/home/matias/WORKSPACE/S2_git/data'    

for e in SelExp:

    data = Folder +'/wavedata'+ str(e)
    wavedata = load_obj(data)

    idx = list(wavedata.keys())

    datapsth = Folder +'/psthdata' + str(e)
    psthdata = load_obj(datapsth)


    for n in idx:
        wavedata[n].update(psthdata[n])    
            
    data = Folder +'data/datamix' + str(e)

    save_obj(wavedata, data)



In [ ]:
idx = list(PSTHdata.keys())
PSTHdata[idx[0]].keys()
#PSTHdata[idx[0]]


In [ ]:
Spikes.keys()

In [ ]:
PSTHdata[idx[0]][1]